# 데이터 준비


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/dacon_cv
!ls

/content/drive/MyDrive/dacon_cv
'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 prediction.csv
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json
 train.json
 val.json


In [3]:
!nvidia-smi

Fri Feb 26 02:10:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
len(os.listdir("./dirty_mnist/"))

In [4]:
!rm -rf dirty_mnist/ 
!mkdir dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 prediction.csv
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json
 train.json
 val.json


In [5]:
!unzip -q dirty_mnist_2nd.zip -d dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 prediction.csv
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json
 train.json
 val.json


In [6]:
!rm -rf dirty_mnist/dirty_mnist_train/

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil
import os
import json

In [8]:
namelist = os.listdir('./dirty_mnist/')
answer = pd.read_csv('dirty_mnist_answer.csv')
train, val = train_test_split(namelist, test_size=0.1, random_state=23)
train_num = [int(t[:5]) for t in train]
val_num = [int(v[:5]) for v in val]
train_ohe = [True if i in train_num else False for i in range(50000)]
val_ohe = [True if i in val_num else False for i in range(50000)]
train_ans = answer[train_ohe]
val_ans = answer[val_ohe]

In [ ]:
%cd ./dirty_mnist/

In [ ]:
os.makedirs('./dirty_mnist_train', exist_ok=True)
os.makedirs('./dirty_mnist_val', exist_ok=True)
for t in train:
    shutil.move(t, './dirty_mnist_train')
for v in val:
    shutil.move(v, './dirty_mnist_val')

In [9]:
def csv_to_coco(df, json_name):
    dic = {
        "info": {
            "description": "DACON DIRTY MNIST DATASET",
            "url": "https://oranz.tistory.com/",
            "version": "1.0",
            "year": 2021,
            "contributor": "DACON",
            "date_created": "2021/02/18"
        },
        "licenses": [{
            "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
            "id": 1,
            "name": "Attribution-NonCommercial-ShareAlike License"
        }],
        "images": [],
        "annotations": [],
        "categories": []
    }
    
    for i in range(1, 27):
        dic["categories"].append({"supercategory": "alphabet", "id": i, "name": chr(i + 96)})
        
    anno_id = 1
    for r in range(len(df)):
        image_id = int(df.iloc[r, 0]) # cast to int for JSON serialization
        name = str(image_id)
        name = '0' * (5 - len(name)) + name + '.png'
        dic["images"].append({
            "license": 1,
            "file_name": name,
            "coco_url": "",
            "height": 256,
            "width": 256,
            "date_captured": "2020-05-19 23:03:57",
            "flickr_url": "",
            "id": image_id
        })
        
        for c in range(1, 27):
            if df.iloc[r, c] == 1:
                dic["annotations"].append({
                    "segmentation": [],
                    "area": 232,
                    "iscrowd": 0,
                    "image_id": image_id,
                    "bbox": [1, 5, 4, 9],
                    "category_id": c,
                    "id": anno_id
                })
                anno_id += 1
    with open(json_name, "w") as json_file:
        json.dump(dic, json_file)

In [10]:
csv_to_coco(val_ans, "val.json")
csv_to_coco(train_ans, "train.json")

# Train

In [11]:
!pip install inplace-abn
!pip install randaugment

     |████████████████████████████████| 143kB 7.2MB/s 
  Created wheel for inplace-abn: filename=inplace_abn-1.1.0-cp37-cp37m-linux_x86_64.whl size=2754614 sha256=06e9537d5d0fe656de669d606c26cc200730cc6a1b9504363099977e86e2a2c4
  Stored in directory: /root/.cache/pip/wheels/90/e6/ce/baadcff0441c600caa5874d4d3322a7909e724fb7abab21a15
Successfully built inplace-abn


In [12]:
%cd ./ASL/

/content/drive/My Drive/dacon_cv/ASL


In [13]:
import argparse
import easydict
import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data.distributed
import torchvision.transforms as transforms
import os
from torch.optim import lr_scheduler
from src.helper_functions.helper_functions import mAP, CocoDetection, CutoutPIL, ModelEma, add_weight_decay
from src.models import create_model
from src.loss_functions.losses import AsymmetricLoss
from randaugment import RandAugment
from torch.cuda.amp import GradScaler, autocast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import imutils
import zipfile
import os
from PIL import Image
from typing import Tuple, Sequence, Callable
import glob
from sklearn.model_selection import KFold
import time
import random
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset

resnext = torch.hub.load('pytorch/vision:v0.6.0', 'resnext50_32x4d', pretrained=True)

class Resnext(nn.Module):
    def __init__(self):
        super(Resnext, self).__init__()
        self.resnext = resnext 
        self.FC = nn.Linear(1000, 26)
        nn.init.xavier_normal_(self.FC.weight)
      
        

    def forward(self, x):
        x = self.resnext(x)
        x = torch.sigmoid(self.FC(x))
        return x

model = Resnext()
model

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


Resnext(
  (resnext): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seque

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = Resnext().to(device)

args = easydict.EasyDict({ "data": "../dirty_mnist", 
                          "num_classes": 26, 
                          "workers": 4, 
                          "image_size": 224, 
                          "thre": 0.8,
                          "batch_size": 128,
                          "print_freq": 64
                          })
args.do_bottleneck_head = False

def validate_multi(val_loader, model):
    print("starting validation")
    Sig = torch.nn.Sigmoid()
    preds_regular = []
    targets = []
    for i, (input, target) in enumerate(val_loader):
        target = target
        target = target.max(dim=1)[0]
        # compute output
        with torch.no_grad():
            with autocast():
                output_regular = Sig(model(input.cuda())).cpu()

        # for mAP calculation
        preds_regular.append(output_regular.cpu().detach())
        targets.append(target.cpu().detach())

    mAP_score_regular = mAP(torch.cat(targets).numpy(), torch.cat(preds_regular).numpy())
    print("mAP score regular {:.2f}".format(mAP_score_regular))
    return mAP_score_regular


# COCO Data loading
instances_path_val = os.path.join("/content/drive/My Drive/dacon_cv", 'val.json')
instances_path_train = os.path.join("/content/drive/My Drive/dacon_cv", 'train.json')
data_path_val   = f'{args.data}/'    # args.data
data_path_train = f'{args.data}/'  # args.data
val_dataset = CocoDetection(data_path_val,
                            instances_path_val,
                            transforms.Compose([
                                transforms.ToTensor(),
                                # normalize, # no need, toTensor does normalization
                            ]))
train_dataset = CocoDetection(data_path_train,
                              instances_path_train,
                              transforms.Compose([
                                  CutoutPIL(cutout_factor=0.5),
                                  transforms.RandomHorizontalFlip(),
                                  transforms.RandomVerticalFlip(),
                                  transforms.ToTensor(),
                                  # normalize,
                              ]))
print("len(val_dataset): ", len(val_dataset))
print("len(train_dataset): ", len(train_dataset))

# Pytorch Data loader
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=args.batch_size, shuffle=True,
    num_workers=args.workers, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=args.batch_size, shuffle=False,
    num_workers=args.workers, pin_memory=False)


Epochs=30
batch_size=64 
lr = 23e-5
weight_decay = 1e-7
criterion = AsymmetricLoss(gamma_neg=4, gamma_pos=1, clip=0.05)
parameters = add_weight_decay(model, weight_decay)
optimizer = torch.optim.Adam(params=parameters, lr=lr, weight_decay=0)  # true wd, filter_bias_and_bn
steps_per_epoch = len(train_loader)
scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=lr, steps_per_epoch=steps_per_epoch, epochs=Epochs,
                                    pct_start=0.2)

highest_mAP = 0
trainInfoList = []
scaler = GradScaler()

for epoch in range(Epochs):
  for i, (inputData, target) in enumerate(train_loader):
      inputData = inputData.cuda()
      target = target.cuda()  # (batch,3,num_classes)
      target = target.max(dim=1)[0]
      with autocast():  # mixed precision
          output = model(inputData).float()  # sigmoid will be done in loss !
      loss = criterion(output, target)
      model.zero_grad()

      scaler.scale(loss).backward()
      # loss.backward()

      scaler.step(optimizer)
      scaler.update()
      # optimizer.step()

      scheduler.step()

      # store information
      if i % 100 == 0:
          trainInfoList.append([epoch, i, loss.item()])
          print('Epoch [{}/{}], Step [{}/{}], LR {:.1e}, Loss: {:.1f}'
                .format(epoch, Epochs, str(i).zfill(3), str(steps_per_epoch).zfill(3),
                        scheduler.get_last_lr()[0], \
                        loss.item()))

  try:
      torch.save(model.state_dict(), os.path.join(
          'resnext_asl/', 'model-{}-{}.pth'.format(epoch + 1, i + 1)))
  except:
      print("model per epoch is not being saved.")
      pass

  model.eval()
  mAP_score = validate_multi(val_loader, model)
  model.train()
  if mAP_score > highest_mAP:
      highest_mAP = mAP_score
      try:
          torch.save(model.state_dict(), os.path.join(
              'resnext_asl/', 'model-highest.pth'))
      except:
          print("best model is not being saved.")
          pass
  print('current_mAP = {:.2f}, highest_mAP = {:.2f}\n'.format(mAP_score, highest_mAP))


def validate_multi(val_loader, model):
    print("starting validation")
    Sig = torch.nn.Sigmoid()
    preds_regular = []
    targets = []
    for i, (input, target) in enumerate(val_loader):
        target = target
        target = target.max(dim=1)[0]
        # compute output
        with torch.no_grad():
            with autocast():
                output_regular = Sig(model(input.cuda())).cpu()

        # for mAP calculation
        preds_regular.append(output_regular.cpu().detach())
        targets.append(target.cpu().detach())

    mAP_score_regular = mAP(torch.cat(targets).numpy(), torch.cat(preds_regular).numpy())
    print("mAP score regular {:.2f}".format(mAP_score_regular))
    return mAP_score_regular


loading annotations into memory...
Done (t=0.15s)
creating index...
index created!
loading annotations into memory...
Done (t=2.34s)
creating index...
index created!
len(val_dataset):  5000
len(train_dataset):  45000


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch [0/30], Step [000/352], LR 9.2e-06, Loss: 489.7
Epoch [0/30], Step [100/352], LR 1.0e-05, Loss: 452.1
Epoch [0/30], Step [200/352], LR 1.4e-05, Loss: 440.2
Epoch [0/30], Step [300/352], LR 2.0e-05, Loss: 439.7
starting validation
mAP score regular 49.73
current_mAP = 49.73, highest_mAP = 49.73

Epoch [1/30], Step [000/352], LR 2.4e-05, Loss: 438.4
Epoch [1/30], Step [100/352], LR 3.3e-05, Loss: 439.0
Epoch [1/30], Step [200/352], LR 4.5e-05, Loss: 439.3
Epoch [1/30], Step [300/352], LR 5.7e-05, Loss: 432.7
starting validation
mAP score regular 54.86
current_mAP = 54.86, highest_mAP = 54.86

Epoch [2/30], Step [000/352], LR 6.5e-05, Loss: 432.9
Epoch [2/30], Step [100/352], LR 7.9e-05, Loss: 428.3
Epoch [2/30], Step [200/352], LR 9.5e-05, Loss: 429.0
Epoch [2/30], Step [300/352], LR 1.1e-04, Loss: 424.2
starting validation
mAP score regular 61.49
current_mAP = 61.49, highest_mAP = 61.49

Epoch [3/30], Step [000/352], LR 1.2e-04, Loss: 423.6
Epoch [3/30], Step [100/352], LR 1.4e-04

KeyboardInterrupt: ignored

# Test


In [ ]:
!rm -rf test_dirty_mnist/
!mkdir test_dirty_mnist/
!unzip -q test_dirty_mnist_2nd.zip -d test_dirty_mnist/
!ls

'제_2회 컴퓨터 비전 학습 경진대회 베이스라인 코드.ipynb'
 ASL
 converT2coco.ipynb
 dirty_mnist
 dirty_mnist_2nd_answer.csv
 dirty_mnist_2nd.zip
 dirty_mnist_answer.csv
 dirty_mnist.zip
 mnist_data
 sample_submission.csv
 test_dirty_mnist
 test_dirty_mnist_2nd.zip
 test.json


In [25]:
import torch
from src.helper_functions.helper_functions import mAP, CocoDetection, CutoutPIL, ModelEma, add_weight_decay
from src.loss_functions.losses import AsymmetricLoss, AsymmetricLossOptimized
import torchvision.transforms as transforms
from src.models import create_model
import argparse
import easydict
import numpy as np
import pandas as pd


# parsing args
args = easydict.EasyDict({ "model_path": "/content/drive/My Drive/dacon_cv/ASL/resnext_asl/model-highest.pth",
                          "model_name": "resnext_asl",
                          "workers": 4, 
                          "dataset_type": "MS-COCO", 
                          "th": 0.8,
                          "num_classes": 26
                          })
instances_path_test = '/content/drive/My Drive/dacon_cv/test.json'
data_path_test = '/content/drive/My Drive/dacon_cv/test_dirty_mnist'
test_dataset = CocoDetection(data_path_test,
                            instances_path_test,
                            transforms.Compose([
                                #transforms.Resize((224, 224)),
                                transforms.ToTensor()
                            ]))

test_data_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=128, shuffle=False,
    num_workers=4, pin_memory=True, drop_last=False)

predictions_list = []
# 배치 단위로 추론
prediction_df = pd.read_csv("/content/drive/My Drive/dacon_cv/sample_submission.csv")
prediction_array = np.zeros([prediction_df.shape[0],
                            prediction_df.shape[1]- 1])


                    
print("test dset: ", test_dataset)

# setup model
print('creating and loading the model...')
model = Resnext().to(device)
model.load_state_dict(torch.load("/content/drive/My Drive/dacon_cv/ASL/resnext_asl/model-highest.pth", map_location='cpu'))
for idx, sample in enumerate(test_data_loader):
    with torch.no_grad():
        # 추론
        model.eval()
        images = sample[0].cuda()
        probs  = model(images)
        probs = probs.cpu().detach().numpy()
        preds = (probs > 0.5)

        # 예측 결과를 
        # prediction_array에 입력
        batch_index = 128 * idx
        prediction_array[batch_index: batch_index + images.shape[0],:] = preds.astype(int)


loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
test dset:  Dataset CocoDetection
    Number of datapoints: 5000
    Root location: /content/drive/My Drive/dacon_cv/test_dirty_mnist
creating and loading the model...


In [26]:
sample_submission = pd.read_csv("/content/drive/My Drive/dacon_cv/sample_submission.csv")
sample_submission.iloc[:,1:] = prediction_array
sample_submission = sample_submission.astype('int32')
sample_submission.to_csv("resnext_asl_5.csv", index = False)
sample_submission

,index,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,50000,1,0,1,0,1,1,0,1,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,1,0,1
1,50001,0,1,1,0,1,0,0,0,1,1,0,1,0,0,1,1,1,1,0,0,1,1,0,0,1,1
2,50002,0,0,0,1,1,0,1,0,1,1,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,1
3,50003,1,1,0,1,0,0,1,0,1,0,1,1,1,0,0,1,1,0,0,1,0,1,0,1,1,1
4,50004,0,0,1,0,1,1,0,0,0,0,0,1,1,0,1,0,1,0,1,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,54995,0,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4996,54996,1,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,1,0,1,0,0,0,0,1,0,1
4997,54997,1,0,0,1,0,1,0,0,0,1,1,0,1,0,1,1,0,0,0,1,1,1,1,0,0,1
4998,54998,0,0,1,0,0,0,1,0,1,1,0,1,0,0,0,0,1,1,0,1,0,0,1,0,0,1
